### 自迴歸解碼 

描述模型不斷生成 Token 的過程。模型生成的 Token 會重新變成模型的輸入，模型在根據這些新生成的 Token 繼續往下生成，直到觸發結束條件為止。

* 相對於非自迴歸 (Non-Autogressive, NAR) 模型而言，其差別在於自迴歸模型會把自己的輸出當成輸入，而 NAR 模型會一次把整個序列生出來。
* 在 LLM 裡面，每次**推論 (Inference)** 只會生成一個 Token，必須要經過多次推論，才能完成完整的文本**生成 (Generation)**。

#### 拆解 model.generate 的背後原理：

* 透過 `.forward()` 來進行推論：
    * 使用 `model.forward(input_ids)` 與 `model(input_ids)` 同樣可以得到模型推論結果。
    * 使用 `.forward()` 當範例，住要是強調該步驟為前向運算，另外這樣的回傳結果也會有比較明確的型別提示。
    * 實際通常不使用 `.forward()` ，因為 PyTorch 中有一些錢處理與後處理的 Hooks ，如果使用它可能會忽略掉這些 Hoooks 導致某些環節處理不正常。
* 產生一個類別為 `CausalLMOutputWithPast` 的輸出，裡面有兩個重要的資訊，分別為 `logits` 與 `past_key_values`。
    * `logits` : 
        * 模型預測下個 Token 的機率表。
        * 可以對 `logits` 做隨機取樣，或者直接計算 `argmax` 做取樣機最高的 Token 做 Greedy Decode。
    * `past_key_values` :
        * 俗稱**鍵值快取 (Key-Values Cache)**
        * 是模型對**已知輸入**的運算結果。
        * 因為自迴歸的特性，所以 Decoder LM 每次生成的輸出都會變成下個回合的輸入，因此這個 KV Cache 會越長越大，同時也是**消耗 GPU 記憶體的元兇之一**。

In [1]:
import torch
from transformers import GenerationConfig
from transformers import TextStreamer
from transformers import BitsAndBytesConfig
from transformers import AutoModelForCausalLM as ModelCls
from transformers import AutoTokenizer as TkCls

model_path = "google/gemma-2b-it"
model: ModelCls = ModelCls.from_pretrained(
    model_path,
    device_map="auto",
    low_cpu_mem_usage=True,
    quantization_config=BitsAndBytesConfig(load_in_8bit=True),
)
tk: TkCls = TkCls.from_pretrained(model_path)

prompt = "[INST] 使用繁體中文回答，請問什麼是大型語言模型？ [/INST] "

tokens = tk(prompt, return_tensors="pt")

input_ids = tokens["input_ids"].to("cuda")

with torch.no_grad():
    outputs = model.forward(input_ids)
    print(outputs)

/home/ai-x/miniconda3/envs/ann_py311/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.16it/s]


CausalLMOutputWithPast(loss={'logits': tensor([[[  3.5840,   7.4258,  49.3125,  ...,  -4.4375,  -0.9116,   3.9941],
         [-23.4688,  -5.3438, -15.2500,  ..., -17.4688, -14.7031, -22.7656],
         [-21.8281,   3.9961,  -3.3242,  ..., -10.1094,  -6.3477, -21.7500],
         ...,
         [-19.7031,   1.8838, -21.7656,  ..., -17.5156, -16.1094, -19.0156],
         [-31.8750,   4.8516, -44.9688,  ..., -28.1875, -24.3281, -31.0938],
         [-34.7188,  -4.0625, -21.8125,  ..., -26.6719, -26.4219, -34.0938]]],
       device='cuda:0', dtype=torch.float16), 'past_key_values': <transformers.cache_utils.DynamicCache object at 0x7ea4487c1450>}, logits=tensor([[[  3.5840,   7.4258,  49.3125,  ...,  -4.4375,  -0.9116,   3.9941],
         [-23.4688,  -5.3438, -15.2500,  ..., -17.4688, -14.7031, -22.7656],
         [-21.8281,   3.9961,  -3.3242,  ..., -10.1094,  -6.3477, -21.7500],
         ...,
         [-19.7031,   1.8838, -21.7656,  ..., -17.5156, -16.1094, -19.0156],
         [-31.8750,   

* 有了這份 KV Cache，就不用每次推論時都把整個 `input_ids` 丟進去，只要留新長出來的 Token 就好，因為舊的輸入都已經被模型 Decode 成 KV Cache 了。
* 下個回合的推論：

In [2]:
with torch.no_grad():
    outputs = model(
        outputs.logits.argmax(-1)[:, -1:],  # Greedy Decode
        past_key_values=outputs.past_key_values,
    )


#### 將以上邏輯整理成一個迴圈來進行：
* 這邊固定進行 16 次推論，中間如果遇到 EOS Token 就會跳出迴圈。
* 拆解生成步驟的好處：
    * 可以針對 `logits` 的機率分佈做觀察。
    * 有些時候可以看到模型在「**說謊**」時，那個部份輸出的 Token 的機率分佈會特別的低，因此也可以用來計算模型的**信心度**之類的。

In [3]:
pkv = None
output_tokens = list()
for i in range(16):
    with torch.no_grad():
        outputs = model.forward(input_ids, past_key_values=pkv)
    
    next_token = outputs.logits.argmax(-1)[:, -1:] # 直接計算 `argmax` 做取樣機最高的 Token 做 Greedy Decode。
    token_id = next_token.item()

    if token_id == tk.eos_token_id:
        break

    output_tokens.append(token_id)
    input_ids = next_token
    pkv = outputs.past_key_values

print(tk.decode(output_tokens))
# 輸出結果：大型語言模型（Large Language Model，LLM）



大型語言模型（LLM）是一種 AI 模型，它能夠像人類


In [4]:
print(outputs.logits)
print(outputs.past_key_values)

tensor([[[-36.7812, -15.7031, -31.0156,  ..., -26.9219, -23.9219, -36.5625]]],
       device='cuda:0', dtype=torch.float16)
